In [54]:
# -*- coding: utf-8 -*-

from __future__ import print_function

import time
import warnings
import numpy as np
import time
import matplotlib.pyplot as plt
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

Using TensorFlow backend.


In [6]:
import numpy as np

In [11]:
filename='E:/machine_data/LSTM-Neural-Network-for-Time-Series-Prediction-master/sp500.csv'
seq_len=50

In [12]:
f = open(filename, 'rb').read()
data = f.decode().split('\n')

In [15]:
print (f[0:30])
print (data[0:3])

b'1455.219971\n1399.420044\n1402.1'
['1455.219971', '1399.420044', '1402.109985']


In [28]:
sequence_length = seq_len + 1
result = []
for index in range(len(data) - sequence_length):
    result.append(data[index: index + sequence_length])  #得到长度为seq_len+1的向量，最后一个作为label

print('result len:',len(result))   #4121
print('result shape:',np.array(result).shape)  #（4121,51）
print(len(result[0]))
print(result[0])

result len: 4121
result shape: (4121, 51)
51
['1455.219971', '1399.420044', '1402.109985', '1403.449951', '1441.469971', '1457.599976', '1438.560059', '1432.25', '1449.680054', '1465.150024', '1455.140015', '1455.900024', '1445.569946', '1441.359985', '1401.530029', '1410.030029', '1404.089966', '1398.560059', '1360.160034', '1394.459961', '1409.280029', '1409.119995', '1424.969971', '1424.369995', '1424.23999', '1441.719971', '1411.709961', '1416.829956', '1387.119995', '1389.939941', '1402.050049', '1387.670044', '1388.26001', '1346.089966', '1352.170044', '1360.689941', '1353.430054', '1333.359985', '1348.050049', '1366.420044', '1379.189941', '1381.76001', '1409.170044', '1391.280029', '1355.619995', '1366.699951', '1401.689941', '1395.069946', '1383.619995', '1359.150024', '1392.140015']


In [29]:
type (result)

list

In [44]:
result = np.array(result)
#划分train、test
row = round(0.9 * result.shape[0])
train = result[:row, :]
np.random.shuffle(train)
x_train = train[:, :-1]
y_train = train[:, -1]
x_test = result[row:, :-1]
y_test = result[row:, -1]

In [45]:
print (row)

3709


In [46]:
print (x_train.shape)

(3709, 50)


In [47]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  

print('X_train shape:',x_train.shape)  #(3709L, 50L, 1L)
print('y_train shape:',y_train.shape)  #(3709L,)
print('X_test shape:',x_test.shape)    #(412L, 50L, 1L)
print('y_test shape:',y_test.shape)    #(412L,)

X_train shape: (3709, 50, 1)
y_train shape: (3709,)
X_test shape: (412, 50, 1)
y_test shape: (412,)


In [48]:
print (x_train[0].shape)
print (x_train[0])

(50, 1)
[['1977.650024']
 ['1963.709961']
 ['1972.829956']
 ['1964.680054']
 ['1967.569946']
 ['1977.099976']
 ['1973.280029']
 ['1981.569946']
 ['1958.119995']
 ['1978.219971']
 ['1973.630005']
 ['1983.530029']
 ['1987.01001']
 ['1987.97998']
 ['1978.339966']
 ['1978.910034']
 ['1969.949951']
 ['1970.069946']
 ['1930.670044']
 ['1925.150024']
 ['1938.98999']
 ['1920.209961']
 ['1920.23999']
 ['1909.569946']
 ['1931.589966']
 ['1936.920044']
 ['1933.75']
 ['1946.719971']
 ['1955.180054']
 ['1955.060059']
 ['1971.73999']
 ['1981.599976']
 ['1986.51001']
 ['1992.369995']
 ['1988.400024']
 ['1997.920044']
 ['2000.02002']
 ['2000.119995']
 ['1996.73999']
 ['2003.369995']
 ['2002.280029']
 ['2000.719971']
 ['1997.650024']
 ['2007.709961']
 ['2001.540039']
 ['1988.439941']
 ['1995.689941']
 ['1997.449951']
 ['1985.540039']
 ['1984.130005']]


In [32]:
def load_data(filename, seq_len):
    f = open(filename, 'rb').read()
    data = f.decode().split('\n')

    print('data len:',len(data))       #4172
    print('sequence len:',seq_len)     #50

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])  #得到长度为seq_len+1的向量，最后一个作为label

    print('result len:',len(result))   #4121
    print('result shape:',np.array(result).shape)  #（4121,51）

    result = np.array(result)

    #划分train、test
    row = round(0.9 * result.shape[0])
    train = result[:row, :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[row:, :-1]
    y_test = result[row:, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  

    print('X_train shape:',x_train.shape)  #(3709L, 50L, 1L)
    print('y_train shape:',y_train.shape)  #(3709L,)
    print('X_test shape:',x_test.shape)    #(412L, 50L, 1L)
    print('y_test shape:',y_test.shape)    #(412L,)

    return [x_train, y_train, x_test, y_test]

In [33]:
[x_train, y_train, x_test, y_test]=load_data('E:/machine_data/LSTM-Neural-Network-for-Time-Series-Prediction-master/sp500.csv', 50)

data len: 4172
sequence len: 50
result len: 4121
result shape: (4121, 51)
X_train shape: (3709, 50, 1)
y_train shape: (3709,)
X_test shape: (412, 50, 1)
y_test shape: (412,)


In [51]:
def build_model(layers):  #layers [1,50,100,1]
    model = Sequential()

    #Stack LSTM
    model.add(LSTM(input_dim=layers[0],output_dim=layers[1],return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(layers[2],return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(output_dim=layers[3]))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop",metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

In [52]:
def predict_point_by_point(model, data):
    predicted = model.predict(data)
    print('predicted shape:',np.array(predicted).shape)  #(412L,1L)
    predicted = np.reshape(predicted, (predicted.size,))
    return predicted

In [55]:
model = build_model([1, 50, 100, 1])

model.fit(x_train,y_train,batch_size=512,epochs=1,validation_split=0.05)

D:\anaconda1\lib\site-packages\ipykernel\__main__.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
D:\anaconda1\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=50, input_shape=(None, 1), return_sequences=True)`
D:\anaconda1\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
D:\anaconda1\lib\site-packages\keras\models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Compilation Time :  0.03515338897705078
Train on 3523 samples, validate on 186 samples
Epoch 1/1
3523/3523 [==============================] - 7s - loss: 1689739.1193 - acc: 0.0000e+00 - val_loss: 1777554.3750 - val_acc: 0.0000e+00


In [57]:
point_by_point_predictions = predict_point_by_point(model, x_test)
print('point_by_point_predictions shape:',np.array(point_by_point_predictions).shape)  #(412L)

predicted shape: (412, 1)
point_by_point_predictions shape: (412,)


In [58]:
def plot_results(predicted_data, true_data, filename):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.show()
    plt.savefig(filename+'.png')

In [59]:
plot_results(point_by_point_predictions,y_test,'point_by_point_predictions')

### 下面是总代码：

In [67]:
# -*- coding: utf-8 -*-

from __future__ import print_function

import time
import warnings
import numpy as np
import time
import matplotlib.pyplot as plt
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

warnings.filterwarnings("ignore")

def load_data(filename, seq_len, normalise_window):
    f = open(filename, 'rb').read()
    data = f.decode().split('\n')

    print('data len:',len(data))
    print('sequence len:',seq_len)

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])  #得到长度为seq_len+1的向量，最后一个作为label

    print('result len:',len(result))
    print('result shape:',np.array(result).shape)
    print(result[:1])

    if normalise_window:
        result = normalise_windows(result)

    print(result[:1])
    print('normalise_windows result shape:',np.array(result).shape)

    result = np.array(result)

    #划分train、test
    row = round(0.9 * result.shape[0])
    train = result[:row, :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[row:, :-1]
    y_test = result[row:, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    return [x_train, y_train, x_test, y_test]

def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:   #window shape (sequence_length L ,)  即(51L,)
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

def build_model(layers):  #layers [1,50,100,1]
    model = Sequential()

    model.add(LSTM(input_dim=layers[0],output_dim=layers[1],return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(layers[2],return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(output_dim=layers[3]))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop",metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

#直接全部预测
def predict_point_by_point(model, data):
    predicted = model.predict(data)
    print('predicted shape:',np.array(predicted).shape)  #(412L,1L)
    predicted = np.reshape(predicted, (predicted.size,))
    return predicted

#滚动预测
def predict_sequence_full(model, data, window_size):  #data X_test
    curr_frame = data[0]  #(50L,1L)
    predicted = []
    for i in range(len(data)):
        #x = np.array([[[1],[2],[3]], [[4],[5],[6]]])  x.shape (2, 3, 1) x[0,0] = array([1])  x[:,np.newaxis,:,:].shape  (2, 1, 3, 1)
        predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])  #np.array(curr_frame[newaxis,:,:]).shape (1L,50L,1L)
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)   #numpy.insert(arr, obj, values, axis=None)
    return predicted

def predict_sequences_multiple(model, data, window_size, prediction_len):  #window_size = seq_len
    prediction_seqs = []
    for i in range(int(len(data)/prediction_len)):
        curr_frame = data[i*prediction_len]
        predicted = []
        for j in range(prediction_len):
            predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    return prediction_seqs

def plot_results(predicted_data, true_data, filename):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.show()
    plt.savefig(filename+'.png')

def plot_results_multiple(predicted_data, true_data, prediction_len):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    #Pad the list of predictions to shift it in the graph to it's correct start
    for i, data in enumerate(predicted_data):
        padding = [None for p in range(i * prediction_len)]
        plt.plot(padding + data, label='Prediction')
        plt.legend()
    plt.show()
    plt.savefig('plot_results_multiple.png')

if __name__=='__main__':
    global_start_time = time.time()
    epochs  = 1
    seq_len = 50

    print('> Loading data... ')

    X_train, y_train, X_test, y_test = load_data('E:/machine_data/LSTM-Neural-Network-for-Time-Series-Prediction-master/sp500.csv', seq_len, True)

    print('X_train shape:',X_train.shape)  #(3709L, 50L, 1L)
    print('y_train shape:',y_train.shape)  #(3709L,)
    print('X_test shape:',X_test.shape)    #(412L, 50L, 1L)
    print('y_test shape:',y_test.shape)    #(412L,)

    print('> Data Loaded. Compiling...')

    model = build_model([1, 50, 100, 1])

    model.fit(X_train,y_train,batch_size=512,nb_epoch=epochs,validation_split=0.05)

    multiple_predictions = predict_sequences_multiple(model, X_test, seq_len, prediction_len=50)
    print('multiple_predictions shape:',np.array(multiple_predictions).shape)   #(8L,50L)

    full_predictions = predict_sequence_full(model, X_test, seq_len)
    print('full_predictions shape:',np.array(full_predictions).shape)    #(412L,)

    point_by_point_predictions = predict_point_by_point(model, X_test)
    print('point_by_point_predictions shape:',np.array(point_by_point_predictions).shape)  #(412L)

    print('Training duration (s) : ', time.time() - global_start_time)

    plot_results_multiple(multiple_predictions, y_test, 50)
    plot_results(full_predictions,y_test,'full_predictions')
    plot_results(point_by_point_predictions,y_test,'point_by_point_predictions')

> Loading data... 
data len: 4172
sequence len: 50
result len: 4121
result shape: (4121, 51)
[['1455.219971', '1399.420044', '1402.109985', '1403.449951', '1441.469971', '1457.599976', '1438.560059', '1432.25', '1449.680054', '1465.150024', '1455.140015', '1455.900024', '1445.569946', '1441.359985', '1401.530029', '1410.030029', '1404.089966', '1398.560059', '1360.160034', '1394.459961', '1409.280029', '1409.119995', '1424.969971', '1424.369995', '1424.23999', '1441.719971', '1411.709961', '1416.829956', '1387.119995', '1389.939941', '1402.050049', '1387.670044', '1388.26001', '1346.089966', '1352.170044', '1360.689941', '1353.430054', '1333.359985', '1348.050049', '1366.420044', '1379.189941', '1381.76001', '1409.170044', '1391.280029', '1355.619995', '1366.699951', '1401.689941', '1395.069946', '1383.619995', '1359.150024', '1392.140015']]
[[0.0, -0.03834466823710192, -0.03649619099406931, -0.03557539137153576, -0.009448743333663301, 0.001635495009297161, -0.011448380541775882, -0.01

In [68]:
print (X_train[0])

[[ 0.        ]
 [-0.01070544]
 [-0.00655728]
 [-0.0165247 ]
 [-0.02102908]
 [-0.01115506]
 [-0.00427532]
 [ 0.0057006 ]
 [ 0.01038308]
 [ 0.00885622]
 [ 0.01274136]
 [ 0.01291946]
 [ 0.00947545]
 [ 0.01593094]
 [ 0.01691499]
 [ 0.0107394 ]
 [ 0.01983306]
 [ 0.02158908]
 [ 0.01457369]
 [ 0.01583764]
 [ 0.01562557]
 [ 0.01342852]
 [ 0.01873884]
 [ 0.01634659]
 [ 0.01864543]
 [ 0.02126673]
 [ 0.0235316 ]
 [ 0.02724712]
 [ 0.03233687]
 [ 0.03160735]
 [ 0.02949511]
 [ 0.02972417]
 [ 0.01856911]
 [ 0.01079873]
 [ 0.01005223]
 [ 0.01928165]
 [ 0.01782261]
 [ 0.01059515]
 [ 0.01323333]
 [ 0.02218284]
 [ 0.01365747]
 [ 0.01614301]
 [ 0.0280106 ]
 [ 0.03444062]
 [ 0.0367904 ]
 [ 0.03770662]
 [ 0.0404296 ]
 [ 0.04163422]
 [ 0.03587428]
 [ 0.04284722]]


### 某家店的某顾客的历史消费的时间推测该顾客前下次来店的时间

In [ ]:
import pandas as pd
import numpy as np

def create_interval_dataset(dataset, look_back):
    """
    :param dataset: input array of time intervals
    :param look_back: each training set feature length
    :return: convert an array of values into a dataset matrix.
    """
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        dataX.append(dataset[i:i+look_back])
        dataY.append(dataset[i+look_back])
    return np.asarray(dataX), np.asarray(dataY)

df = pd.read_csv("path-to-your-time-interval-file")    
dataset_init = np.asarray(df)    # if only 1 column
dataX, dataY = create_interval_dataset(dataset, lookback=3)    # look back if the training set sequence length

In [ ]:
import pandas as pd
import numpy as np
import random
from keras.models import Sequential, model_from_json
from keras.layers import Dense, LSTM, Dropout

class NeuralNetwork():
    def __init__(self, **kwargs):
        """
        :param **kwargs: output_dim=4: output dimension of LSTM layer; activation_lstm='tanh': activation function for LSTM layers; activation_dense='relu': activation function for Dense layer; activation_last='sigmoid': activation function for last layer; drop_out=0.2: fraction of input units to drop; np_epoch=10, the number of epoches to train the model. epoch is one forward pass and one backward pass of all the training examples; batch_size=32: number of samples per gradient update. The higher the batch size, the more memory space you'll need; loss='mean_square_error': loss function; optimizer='rmsprop'
        """
        self.output_dim = kwargs.get('output_dim', 8)
        self.activation_lstm = kwargs.get('activation_lstm', 'relu')
        self.activation_dense = kwargs.get('activation_dense', 'relu')
        self.activation_last = kwargs.get('activation_last', 'softmax')    # softmax for multiple output
        self.dense_layer = kwargs.get('dense_layer', 2)     # at least 2 layers
        self.lstm_layer = kwargs.get('lstm_layer', 2)
        self.drop_out = kwargs.get('drop_out', 0.2)
        self.nb_epoch = kwargs.get('nb_epoch', 10)
        self.batch_size = kwargs.get('batch_size', 100)
        self.loss = kwargs.get('loss', 'categorical_crossentropy')
        self.optimizer = kwargs.get('optimizer', 'rmsprop')

        def NN_model(self, trainX, trainY, testX, testY):
        """
        :param trainX: training data set
        :param trainY: expect value of training data
        :param testX: test data set
        :param testY: epect value of test data
        :return: model after training
        """
        print "Training model is LSTM network!"
        input_dim = trainX[1].shape[1]
        output_dim = trainY.shape[1] # one-hot label
        # print predefined parameters of current model:
        model = Sequential()
        # applying a LSTM layer with x dim output and y dim input. Use dropout parameter to avoid overfitting
        model.add(LSTM(output_dim=self.output_dim,
                       input_dim=input_dim,
                       activation=self.activation_lstm,
                       dropout_U=self.drop_out,
                       return_sequences=True))
        for i in range(self.lstm_layer-2):
            model.add(LSTM(output_dim=self.output_dim,
                       input_dim=self.output_dim,
                       activation=self.activation_lstm,
                       dropout_U=self.drop_out,
                       return_sequences=True))
        # argument return_sequences should be false in last lstm layer to avoid input dimension incompatibility with dense layer
        model.add(LSTM(output_dim=self.output_dim,
                       input_dim=self.output_dim,
                       activation=self.activation_lstm,
                       dropout_U=self.drop_out))
        for i in range(self.dense_layer-1):
            model.add(Dense(output_dim=self.output_dim,
                        activation=self.activation_last))
        model.add(Dense(output_dim=output_dim,
                        input_dim=self.output_dim,
                        activation=self.activation_last))
        # configure the learning process
        model.compile(loss=self.loss, optimizer=self.optimizer, metrics=['accuracy'])
        # train the model with fixed number of epoches
        model.fit(x=trainX, y=trainY, nb_epoch=self.nb_epoch, batch_size=self.batch_size, validation_data=(testX, testY))
        # store model to json file
        model_json = model.to_json()
        with open(model_path, "w") as json_file:
            json_file.write(model_json)
        # store model weights to hdf5 file
        if model_weight_path:
            if os.path.exists(model_weight_path):
                os.remove(model_weight_path)
            model.save_weights(model_weight_path) # eg: model_weight.h5
        return model